In [ ]:
# ╔═══════════════════════════════════════════════════════════════════════════╗
# ║  DATASET EXAMINER - Analyze Your Revised Dataset                           ║
# ║  Run this BEFORE training to understand your data structure                ║
# ╚═══════════════════════════════════════════════════════════════════════════╝

# ================= CELL 1: SETUP =================

import os
from collections import defaultdict
from google.colab import drive

print("Mounting Drive...")
drive.mount('/content/drive')

# ================= CELL 2: CONFIGURATION =================

# UPDATE THIS PATH to your dataset location
DATASET_ROOT = '/content/drive/MyDrive/Dataset'

print(f"\n📁 Dataset Root: {DATASET_ROOT}")

# ================= CELL 3: EXAMINE DATASET =================

def examine_dataset(root_path):
    """
    Thoroughly examine dataset structure and report findings.
    """

    print("\n" + "=" * 80)
    print("DATASET EXAMINATION REPORT")
    print("=" * 80)

    # Check if root exists
    if not os.path.exists(root_path):
        print(f"\n❌ ERROR: Path does not exist: {root_path}")
        return

    # ─────────────────────────────────────────────────────────────────────────
    # 1. TOP-LEVEL STRUCTURE
    # ─────────────────────────────────────────────────────────────────────────
    print("\n" + "─" * 60)
    print("1. TOP-LEVEL STRUCTURE")
    print("─" * 60)

    top_level = os.listdir(root_path)
    print(f"\nFolders/Files in {root_path}:")
    for item in sorted(top_level):
        item_path = os.path.join(root_path, item)
        if os.path.isdir(item_path):
            print(f"  📁 {item}/")
        else:
            print(f"  📄 {item}")

    # ─────────────────────────────────────────────────────────────────────────
    # 2. CLASS FOLDERS
    # ─────────────────────────────────────────────────────────────────────────
    print("\n" + "─" * 60)
    print("2. CLASS FOLDERS ANALYSIS")
    print("─" * 60)

    class_folders = [f for f in top_level if os.path.isdir(os.path.join(root_path, f))]
    print(f"\nDetected class folders: {class_folders}")

    # ─────────────────────────────────────────────────────────────────────────
    # 3. YEAR SUBFOLDERS
    # ─────────────────────────────────────────────────────────────────────────
    print("\n" + "─" * 60)
    print("3. YEAR SUBFOLDERS")
    print("─" * 60)

    structure = {}

    for class_folder in class_folders:
        class_path = os.path.join(root_path, class_folder)
        subfolders = os.listdir(class_path)
        structure[class_folder] = {
            'years': [],
            'files_by_year': {},
            'total_files': 0
        }

        print(f"\n📁 {class_folder}/")

        for item in sorted(subfolders):
            item_path = os.path.join(class_path, item)
            if os.path.isdir(item_path):
                # Year folder
                year_files = [f for f in os.listdir(item_path) if f.lower().endswith(('.tif', '.tiff'))]
                structure[class_folder]['years'].append(item)
                structure[class_folder]['files_by_year'][item] = len(year_files)
                structure[class_folder]['total_files'] += len(year_files)
                print(f"    📁 {item}/ → {len(year_files)} .tif files")
            else:
                # Direct file in class folder
                if item.lower().endswith(('.tif', '.tiff')):
                    structure[class_folder]['total_files'] += 1

    # ─────────────────────────────────────────────────────────────────────────
    # 4. FILE NAMING ANALYSIS
    # ─────────────────────────────────────────────────────────────────────────
    print("\n" + "─" * 60)
    print("4. FILE NAMING ANALYSIS")
    print("─" * 60)

    all_files_info = defaultdict(lambda: {
        'months': set(),
        'ids': set(),
        'sample_files': [],
        'naming_patterns': set()
    })

    for class_folder in class_folders:
        class_path = os.path.join(root_path, class_folder)

        # Walk through all subdirectories
        for root, dirs, files in os.walk(class_path):
            for f in files:
                if not f.lower().endswith(('.tif', '.tiff')):
                    continue

                # Store sample files
                if len(all_files_info[class_folder]['sample_files']) < 10:
                    all_files_info[class_folder]['sample_files'].append(f)

                # Parse filename
                name = f.replace('.tif', '').replace('.TIF', '').replace('.tiff', '')
                parts = name.split('_')

                all_files_info[class_folder]['naming_patterns'].add(len(parts))

                if len(parts) >= 3:
                    # Expected: Class_ID_Month
                    all_files_info[class_folder]['ids'].add(parts[1])
                    all_files_info[class_folder]['months'].add(parts[-1])

    for class_folder in class_folders:
        info = all_files_info[class_folder]
        print(f"\n📁 {class_folder}:")
        print(f"    Sample files: {info['sample_files'][:5]}")
        print(f"    Unique IDs found: {sorted(info['ids'])[:10]}{'...' if len(info['ids']) > 10 else ''}")
        print(f"    ID count: {len(info['ids'])}")
        print(f"    Months found: {sorted(info['months'])}")
        print(f"    Naming pattern (parts): {info['naming_patterns']}")

    # ─────────────────────────────────────────────────────────────────────────
    # 5. DETAILED FILE COUNT
    # ─────────────────────────────────────────────────────────────────────────
    print("\n" + "─" * 60)
    print("5. DETAILED FILE COUNT")
    print("─" * 60)

    print(f"\n{'Class':<15} {'Year':<10} {'Files':<10} {'Locations':<12} {'Months':<10}")
    print("-" * 60)

    total_files = 0
    for class_folder in sorted(class_folders):
        class_path = os.path.join(root_path, class_folder)

        for year in sorted(structure[class_folder]['years']):
            year_path = os.path.join(class_path, year)
            files = [f for f in os.listdir(year_path) if f.lower().endswith(('.tif', '.tiff'))]

            # Count unique locations
            locations = set()
            months = set()
            for f in files:
                parts = f.replace('.tif', '').split('_')
                if len(parts) >= 3:
                    locations.add(parts[1])
                    months.add(parts[-1])

            print(f"{class_folder:<15} {year:<10} {len(files):<10} {len(locations):<12} {len(months):<10}")
            total_files += len(files)

    print("-" * 60)
    print(f"{'TOTAL':<15} {'':<10} {total_files:<10}")

    # ─────────────────────────────────────────────────────────────────────────
    # 6. MONTHS VERIFICATION
    # ─────────────────────────────────────────────────────────────────────────
    print("\n" + "─" * 60)
    print("6. MONTHS VERIFICATION BY CLASS")
    print("─" * 60)

    for class_folder in sorted(class_folders):
        months = all_files_info[class_folder]['months']
        print(f"\n{class_folder}: {sorted(months)}")

    # ─────────────────────────────────────────────────────────────────────────
    # 7. ID RANGE VERIFICATION
    # ─────────────────────────────────────────────────────────────────────────
    print("\n" + "─" * 60)
    print("7. ID RANGE VERIFICATION")
    print("─" * 60)

    for class_folder in sorted(class_folders):
        ids = all_files_info[class_folder]['ids']
        if ids:
            # Try to convert to integers for range analysis
            try:
                int_ids = sorted([int(i) for i in ids])
                print(f"\n{class_folder}:")
                print(f"    ID range: {min(int_ids):03d} to {max(int_ids):03d}")
                print(f"    Total unique IDs: {len(int_ids)}")

                # Check for gaps
                expected = set(range(min(int_ids), max(int_ids) + 1))
                missing = expected - set(int_ids)
                if missing:
                    print(f"    ⚠️ Missing IDs: {sorted(missing)[:10]}{'...' if len(missing) > 10 else ''}")
                else:
                    print(f"    ✓ No gaps in ID sequence")
            except:
                print(f"\n{class_folder}: IDs are not numeric - {sorted(ids)[:10]}")

    # ─────────────────────────────────────────────────────────────────────────
    # 8. COMPLETENESS CHECK
    # ─────────────────────────────────────────────────────────────────────────
    print("\n" + "─" * 60)
    print("8. COMPLETENESS CHECK")
    print("─" * 60)

    for class_folder in sorted(class_folders):
        class_path = os.path.join(root_path, class_folder)

        for year in sorted(structure[class_folder]['years']):
            year_path = os.path.join(class_path, year)
            files = [f for f in os.listdir(year_path) if f.lower().endswith(('.tif', '.tiff'))]

            # Group by location
            location_months = defaultdict(set)
            for f in files:
                parts = f.replace('.tif', '').split('_')
                if len(parts) >= 3:
                    loc_id = parts[1]
                    month = parts[-1]
                    location_months[loc_id].add(month)

            # Check for incomplete locations
            incomplete = []
            expected_months = len(all_files_info[class_folder]['months'])

            for loc_id, months in location_months.items():
                if len(months) < expected_months:
                    incomplete.append((loc_id, len(months)))

            if incomplete:
                print(f"\n⚠️ {class_folder}/{year} - Incomplete locations:")
                for loc_id, count in sorted(incomplete)[:5]:
                    print(f"    ID {loc_id}: only {count}/{expected_months} months")
                if len(incomplete) > 5:
                    print(f"    ... and {len(incomplete) - 5} more")
            else:
                print(f"\n✓ {class_folder}/{year} - All locations complete ({len(location_months)} locations × {expected_months} months)")

    # ─────────────────────────────────────────────────────────────────────────
    # 9. SUMMARY
    # ─────────────────────────────────────────────────────────────────────────
    print("\n" + "=" * 80)
    print("SUMMARY")
    print("=" * 80)

    print(f"""
Dataset Root: {root_path}

Classes Found: {', '.join(sorted(class_folders))}

Structure:
""")

    for class_folder in sorted(class_folders):
        years = structure[class_folder]['years']
        total = structure[class_folder]['total_files']
        months = sorted(all_files_info[class_folder]['months'])
        ids = len(all_files_info[class_folder]['ids'])

        print(f"  {class_folder}:")
        print(f"    Years: {', '.join(sorted(years))}")
        print(f"    Months: {', '.join(months)}")
        print(f"    Unique locations: {ids}")
        print(f"    Total files: {total}")
        print()

    print(f"Grand Total: {total_files} files")

    # ─────────────────────────────────────────────────────────────────────────
    # 10. RECOMMENDATIONS
    # ─────────────────────────────────────────────────────────────────────────
    print("\n" + "=" * 80)
    print("CONFIGURATION FOR TRAINING CODE")
    print("=" * 80)

    # Detect months for each class
    print("\nDetected configuration:")
    print(f"""
DATASET_ROOT = '{root_path}'

CLASS_NAMES = {sorted(class_folders)}

MONTHS_BY_CLASS = {{""")

    for class_folder in sorted(class_folders):
        months = sorted(all_files_info[class_folder]['months'])
        print(f"    '{class_folder}': {months},")

    print("}")

    return structure, all_files_info

# ================= RUN EXAMINATION =================

structure, file_info = examine_dataset(DATASET_ROOT)

Mounting Drive...
Mounted at /content/drive

📁 Dataset Root: /content/drive/MyDrive/Dataset

DATASET EXAMINATION REPORT

────────────────────────────────────────────────────────────
1. TOP-LEVEL STRUCTURE
────────────────────────────────────────────────────────────

Folders/Files in /content/drive/MyDrive/Dataset:
  📁 Other/
  📁 Rice/
  📁 Wheat/

────────────────────────────────────────────────────────────
2. CLASS FOLDERS ANALYSIS
────────────────────────────────────────────────────────────

Detected class folders: ['Wheat', 'Rice', 'Other']

────────────────────────────────────────────────────────────
3. YEAR SUBFOLDERS
────────────────────────────────────────────────────────────

📁 Wheat/
    📁 2023/ → 600 .tif files
    📁 2024/ → 600 .tif files

📁 Rice/
    📁 2023/ → 648 .tif files
    📁 2024/ → 648 .tif files

📁 Other/
    📁 2023/ → 600 .tif files
    📁 2024/ → 600 .tif files

────────────────────────────────────────────────────────────
4. FILE NAMING ANALYSIS
────────────────────